# Week 9 - Macro Basics (Part 8)

###  Ways to delete user-defined macro variables using the

* %SYMDEL statement in open code
* %SYMDEL statement in a macro from
    * Dictionary.Macros
    * SASHELP.VMACRO (automatically generated view that contains a list all macro variables existing in the current session as well as those defined by the SAS System)
    


### Create a macro variable in OPEN CODE

In [4]:
%LET course = STAT6197;
%put &=course;
%put _user_;


11                                                         The SAS System                              13:22 Thursday, March 6, 2025

100        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
100      ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
101        
102        %LET course = STAT6197;
103        %put &=course;
COURSE=STAT6197
104        %put _user_;
GLOBAL COURSE STAT6197
GLOBAL SYS_SQL_IP_ALL -1
GLOBAL SYS_SQL_IP_STMT 
105        
106        
107        ods html5 (id=saspy_internal) close;ods listing;
108        

12                                                         The SAS System                              13:22 Thursday, March 6, 2025

109        


### Delete the macro variable created above using %SYMDEL statement

In [6]:
%SYMDEL course /nowarn;
%put _user_;


15                                                         The SAS System                              13:22 Thursday, March 6, 2025

123        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
123      ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
124        
125        %SYMDEL course /nowarn;
126        %put _user_;
GLOBAL SYS_SQL_IP_ALL -1
GLOBAL SYS_SQL_IP_STMT 
127        
128        
129        ods html5 (id=saspy_internal) close;ods listing;
130        

16                                                         The SAS System                              13:22 Thursday, March 6, 2025

131        


### Create a vertical list of macro variables and display them

In [ ]:
options nodate nonumber nonotes nosource;
ods html close;
%let Put_title = List of Values into a Series of Macro Variables;
proc sql noprint;
 select distinct make
        INTO :makes1-
 FROM SASHELP.CARS ;
 %put Number of Rows: &sqlobs;
quit;
%macro runit;
 %put &Put_title;
 %Do i=1 %TO &Sqlobs;
    %put &&makes&i;
  %end;
%mend runit;
%runit


[Deleting global macro variables by Jim Simon](https://blogs.sas.com/content/sastraining/2018/05/07/deleting-global-macro-variables/)

In [3]:
* Code obtanied from the above source;
ods html close;
%macro deleteALL;
 options nonotes;
 %local vars;
 proc sql noprint;
        select name into: vars separated by ' '
           from dictionary.macros /* DICTIONARY.MACROS is a metadata table 
                                   with one row per macro variable.*/
   where scope='GLOBAL' 
   and not name contains 'SYS_SQL_IP_';
   quit;
    %symdel &vars;  /* The %SYMDEL statement deletes all user-defined global 
                    macro variables.*/
    options notes;
     %put NOTE: Macro variables deleted.;
 %mend deleteALL;
 %deleteALL
 %put _user_;


9                                                          The SAS System                              13:22 Thursday, March 6, 2025

72         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
72       ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
73         
74         * Code obtanied from the above source;
75         ods html close;
76         %macro deleteALL;
77          options nonotes;
78          %local vars;
79          proc sql noprint;
80                 select name into: vars separated by ' '
81                    from dictionary.macros /* DICTIONARY.MACROS is a metadata table
82                                            with one row per macro variable.*/
83            where scope='GLOBAL'
84            and not name contains 'SYS_SQL_IP_';
85            quit;
86             %symdel &vars;  /* The %SYMDEL statement deletes all user-defined global
87           

### Recreate a vertical list of macro variables and display them

In [ ]:
options nodate nonumber nonotes nosource;
ods html close;
%let Put_title = List of Values into a Series of Macro Variables;
proc sql noprint;
 select distinct make
        INTO :makes1-
 FROM SASHELP.CARS ;
 %put Number of Rows: &sqlobs;
quit;


### DESCRIBE TABLE statement with PROC SQL

Since you have recreated the macro variables, you can run the following code to list the names of the columns stored in the DICTIONARY.MACROS entry. 

In [ ]:
options nodate nonumber nonotes nosource;
ods html close;
proc sql;
  describe table dictionary.macros;
quit;

### How to print the data values of the MACROS entry

* Make a SAS data set containing the extract of the Macro entry from the current session using PROC SQL CREATE table

* List the SAS data set using PROC PRINT


In [ ]:
options nodate nonumber nonotes nosource;
ods html close;

proc sql;
create table work.MyMacros as
select * from dictionary.macros;
where name ='GLOBAL';
quit;
proc print data=work.MyMacros;
Title 'Work.Mymacros';
run;

### SASHELP.VMACRO

* Use SASHELP.VMACRO to keep track of all macro variables you have created

[Surviving the SAS® Macro Jungle by Using Your Own Programming Toolkit by
Kevin Russell](http://support.sas.com/resources/papers/387699_macro-programming-tools.pdf)


In [ ]:
proc print data=sashelp.vmacro(where=(name=:'MAKE'));
run;

### SASHELP.VMACRO and %SYMDEL Statement
Kevin Russell (2016) uses the following code to delete all user-defined global macro variables.

In [ ]:
options nodate nonumber nonotes nosource;
ods html close;
%macro delvars;
data vars;
 set sashelp.vmacro;
 run;
data _null_;
 set vars;
 temp=lag(name);
 if scope='GLOBAL' and substr(name,1,3) ne 'SYS' and temp ne name then
 rc=dosubl('%symdel '||trim(left(name))||';');
 run;
%mend;
%delvars
%put _user_;

[A Hands-on Introduction to SAS® Metadata DICTIONARY Tables
and SASHELP Views](https://www.mwsug.org/proceedings/2018/HW/MWSUG-2018-HW-9.pdf)

In [ ]:
*https://blogs.sas.com/content/sastraining/2018/05/07/deleting-global-macro-variables/;
%macro deleteALL;
   options nonotes;
   %local vars;
   proc sql noprint;
           select name into: vars separated by ' '
           from dictionary.macros
                 where scope='GLOBAL' 
     and not name contains 'SYS_SQL_IP_';
   quit;
   %symdel &vars;
   options notes;
   %put NOTE: Macro variables deleted.;
%mend deleteALL;
%deleteAll

The macro below deletes all global macro variables except those containing 'SYS_SQL_IP_'. SAS system-defined macro variables and those dynamically created by SQL queries should be preserved.

In [ ]:
%macro deleteALL; /* Defines the macro deleteALL.*/
    options nonotes; /* Suppresses log notes to keep the output clean. */
    %local varCount i varName; /* Declares macro variables used within the macro */
    
    /* Count the number of global macro variables */
    proc sql noprint;
        select count(*) into :varCount
                  /* The macro variable varCount → Stores the number of 
                     global macro variables to delete.*/
        from dictionary.macros
        where scope = 'GLOBAL' and not name contains 'SYS_SQL_IP_';
    quit;
    
    /* Loop through and delete each macro variable individually */
    %if &varCount > 0 %then %do;Filters only global macro variables
        proc sql noprint; /* Runs an SQL query without displaying output in the log */
            select name into :varName1-:varName&varCount
              from dictionary.macros
               /* Stores macro variable names retrieved from dictionary.macros.*/
            where scope = 'GLOBAL' and not name contains 'SYS_SQL_IP_';
             /* Filters only global macro variables, excluding system-generated 
             SQL macro variables.*/
        quit;

        %do i = 1 %to &varCount;
            %symdel &&varName&i;                      
        %end;
        
        /* Iterates from 1 to varCount.
        &&varName&i → Resolves dynamically to varName1, varName2, etc.
        %symdel &&varName&i; → Deletes each macro variable individually.
        */
    %end;

    options notes;
    %put NOTE: Macro variables deleted.;
%mend deleteALL;

%deleteALL;


#### How the above program works

* Finds all global macro variables (except those containing 'SYS_SQL_IP_').
* Counts how many need to be deleted.
* Retrieves their names into multiple macro variables.
* Loops through and deletes each one using %symdel.
* Prints a message confirming deletion.
